## Template para o cálculo das velocidades e acelerações em cadeias não impostas
---
### A função ''MecSolve'' Calcula as velocidades e acelerações
- Sintaxe:
    MecSolve($<$coords_generalizadas$>$, $<$equações_de_restrição$>$, $<$array_de_velocidade$>$)
   * onde:
       - $<$coords_generalizadas$>$ $\quad\,\,\,\,\, \rightarrow$ Lista contendo as coordenadas generalizadas
       - $<$equações_de_restrição$>$ $\quad\,\, \rightarrow$ Lista contendo as equações de restrição
       - $<$coeficientes_de_velocidade$>$ $\rightarrow$ Lista contendo as velocidades das coordenadas (variáveis correspondentes)
       
       
- A célula a seguir só deve ser rodada uma única vez, no início e, **em hipótese alguma** deve ser modificada

In [ ]:
using SymPy

function MecSolve(c,eᵣ,cₜ)
    f = size(c)[1]-size(eᵣ)[1]
    Cₛ = Array{Sym}([ i for i in c[f+1:end] ])
    Cₚ = Array{Sym}([ i for i in c[1:f] ])
    Eq = Array{Sym}(eᵣ)
    Cₛₜ = Array{Sym}([ i for i in cₜ[f+1:end] ])
    Cₚₜ = Array{Sym}([ i for i in cₜ[1:f] ])
    
    J = Eq.jacobian(Cₛ); F = Eq.jacobian(Cₚ)
    K = -(J^-1)*F; K = Array{Sym}([ simplify(i) for i in K ])
    L = []
    for i in 1:f
        Lprov = K.col(i-1).jacobian(Cₚ)*Cₚₜ + K.col(i-1).jacobian(Cₛ)*Cₛₜ
        push!(L,Lprov)
    end
    L = [simplify(L[i].col(0)[j]) for j in 1:2 , i in 1:f]
    K, L
end

### Utilização do template:
   * Primeiramente devem ser definidas as variáveis simbólicas a serem utilizadas, tais como constantes representativas dos comprimentos das barras, coordenadas generalizadas e as variávesl representativas das velociades;
   * Em seguida devem ser definidas as equações de restrição no formato:
        $$f_1 = f_1(q_1,q_2,\dots,q_m,s_1,s_2,s_3,\dots,s_n)\\
        f_2 = f_2(q_1,q_2,\dots,q_m,s_1,s_2,s_3,\dots,s_n)\\
                  \vdots \\
        f_n = f_n(q_1,q_2,\dots,q_m,s_1,s_2,s_3,\dots,s_n)$$
   * Por fim, devem ser montadas **três listas** contendo as "coordenadas generlizadas", as "linhas das equações de restrição" e as variáveis que irão receber as velocidades principais e secundárias;

Estas três listas devem, então, entrar como argumento da função "MecSolve" (três argumentos apenas). "MecSolve" irá retornar duas matrizes, sendo a primeira a matriz dos ceoficientes de velocidade (**K**) e a segunda a matriz dos coeficientes da aceleração (**L**), em outras palavras, a função "MecSolve" deve ser chamada na seguinte forma:
$$\text{K, L = MecSolve(CoordsGen, EqRest, Ct)}$$
#### Exemplo de Uso
A cadeia do tipo Quadrilátero Biela, da figura abaixo, vai servir como exemplo na criação das variáveis e uso da função acima.

![Cadeia Quadrilátero Biela](https://dc547.4shared.com/img/kN7lEkwsea/172c432f388/quadrilatero.png)

Vejamos então como fica a codificação na célula abaixo.

In [ ]:
a,b,c = symbols("a b c")
θ₁,θₜ₁,θₜₜ₁ = symbols("theta_1 \\dot{\\theta_1} \\ddot{\\theta_1}")
θ₂,θₜ₂,θₜₜ₂ = symbols("theta_2 \\dot{\\theta_2} \\ddot{\\theta_2}")
φ,x,φₜ,ẋ = symbols("varphi x \\dot{\\varphi} \\dot{x}")

f₁ = a*cos(θ₁) + x*cos(φ) - b*cos(θ₂) - c
f₂ = a*sin(θ₁) + x*sin(φ) - b*sin(θ₂)

C = [θ₁,θ₂,φ,x]
Cₜ = [θₜ₁,θₜ₂,φₜ,ẋ]
Eq = [f₁,f₂]

K,L = MecSolve(C,Eq,Cₜ)

display(K*[θₜ₁,θₜ₂]);  display(K*[θₜₜ₁,θₜₜ₂] + L*[θₜ₁,θₜ₂])